Laura Boltà Ballesteros, NIU: 1705130

## __Lab 1: Correlation Attacks in Stream Ciphers__
---

In [29]:
import scipher
plaintext = scipher.read_string_from_file("07-known-plaintext.txt")
ciphertext = scipher.read_string_from_file("07-known-ciphertext.txt")
secret_ciphertext = scipher.read_string_from_file("07-secret-ciphertext.txt")

### __Exercise 1: Find potential correlations__
We want to analyze our stream cipher looking for possible vulnerabilities. Determine whether there is a correlation between the individual outputs of the LFSRs and the keystream. That is, determine if there is a correlation between any of the LFSRs separately and the keystream.

__(a) Explain how you find these possible correlations analytically.__

> There are several ways to find the possible correlations between the individual outputs of the LFSRs and the keystream.
>
> From the given equation $s(t)=(x_1(t) ∧ x_2(t)) ⊕ (¬x_1(t) ∧ x_3(t))$ we can deduce that:
> - When $x_1=1$, $s=x_2$
> - When $x_1=0$, $s=x_3$
>
> Thus, the output $s(t)$ will depend on $x_2$ half of the time and on $x_3$ the other half.
> To verify whether these inputs are correlated with the output, we analyze the truth table: 
>
> | $x_1$ | $x_2$ | $x_3$ | ($x_1 ∧ x_2$) | ($x_1 ∧ x_3$) | $s(t)$ |
> |----|----|----|------------|-------------|------|
> | 0  | 0  | 0  | 0          | 0           | 0    |
> | 0  | 0  | 1  | 0          | 1           | 1    |
> | 0  | 1  | 0  | 0          | 0           | 0    |
> | 0  | 1  | 1  | 0          | 1           | 1    |
> | 1  | 0  | 0  | 0          | 0           | 0    |
> | 1  | 0  | 1  | 0          | 0           | 0    |
> | 1  | 1  | 0  | 1          | 0           | 1    |
> | 1  | 1  | 1  | 1          | 0           | 1    |
>
> From this can also deduce that:
>
> - $s = x_1 \Rightarrow \frac{1}{2}$
>
> - $s = x_2 \Rightarrow \frac{3}{4}$
>
> - $s = x_3 \Rightarrow \frac{3}{4}$
>
> After deducing, we know that more than half of the times the output will be $x_2$. We also know that the same happens for $x_3$. So, we can conlude that for LFSR2 and LFSR3, the keystream equals their output $75\%$ of the time, meaning there is correlation between them and the keystream.
>
> For LFSR1, the probability is exactly $\frac{1}{2}$, meaning it is not correlated (it is random).
>
> So, LFSR2 and LFSR3 are correlated with the keystream and can be potential points of attack, while LFSR1 shows no exploitable correlation.

__(b) Explain how you validate them empirically.__

> To empirically validate it we could:
>
> 1. Generate random bit sequences for $x_1$, $x_2$, and $x_3$.
>
> 2. For each time step, compute $s(t)=(x_1∧x_2) ⊕ (¬x_1∧x_3)$.
>
> 3. Compare $s$ with each input, and keep track of whether $s=x_1, s=x_2, s=x_3$.
>
> 4. Over many samples, compute the match rate for each ($\frac{matches}{number of time steps}$) and check if they are close to the analytical values we got ($\frac{1}{2}$ for $x_1$ and $\frac{3}{4}$ for $x_2$ and $x_3$).

__(c) Provide or identify the code used to do it.__

In [30]:
import random
def rand_nonzero_bits(n): 
    rand_bit_seq = [] 
    for i in range(n): 
        rand_bit_seq.append(random.getrandbits(1)) 
    if not any(rand_bit_seq):
        rand_bit_seq[0] = 1
    return rand_bit_seq

# 1) generate random bit sequences
s1 = rand_nonzero_bits(scipher.Cipher.deg1) # as many bits as the degree of the polynomial
s2 = rand_nonzero_bits(scipher.Cipher.deg2)
s3 = rand_nonzero_bits(scipher.Cipher.deg3)

# 2) to validate that the sequence is valid and to start the LFSRs
cipher = scipher.Cipher((s1, s2, s3))

# 3) generate bits and compare (match rates)
N = 100000
m1 = m2 = m3 = 0
for i in range(N):
    x1 = cipher.l1.clock()
    x2 = cipher.l2.clock()
    x3 = cipher.l3.clock()
    s = (x1 & x2) ^ ((1 - x1) & x3)
    # sometimes s can equal more than one (x1, x2, x3) that's why no 'else'
    if s == x1: 
        m1 += 1
    if s == x2: 
        m2 += 1
    if s == x3: 
        m3 += 1


rate1 = m1 / N   # expect 0.5
rate2 = m2 / N   # expect 0.75
rate3 = m3 / N   # expect 0.75
print(rate1, rate2, rate3)

0.47528 0.76759 0.73466


---

### __Exercise 2: Recover the keystream of the known ciphertext__

Attempt to reconstruct the maximal portion of the keystream employed to encrypt `NN-known-ciphertext.txt`.

__(a) Explain how you recovered the keystream.__
>
> In order to recover the keystream we have analyzed the schem where we can see that:
> - $Keystream ⊕ Plaintext = Ciphertext$.
>
> We know the plaintext and its respective ciphertext, so we can calculate the keystream like this:
> - $Keystream = Plaintext ⊕ Ciphertext$


__(b) Provide or identify the code used to do it.__

In [31]:
def bytes_to_bits(data: bytes) -> list[int]:
    bits = []
    for byte in data:
        for b in f"{byte:08b}":
            bits.append(int(b))
    return bits

# 1) since _xor_bytes works with bytes we need to convert str to bytes and hex to bytes
plaintext_bytes  = plaintext.encode("utf-8")
cihertext_bytes  = bytes.fromhex(ciphertext)

# 2) now we can do the XOR
keystream_bytes  = scipher._xor_bytes(plaintext_bytes, cihertext_bytes)  
# in bites
keystream_bits = bytes_to_bits(keystream_bytes)
print("The keystream is:",keystream_bytes)
print("The keystream is:",keystream_bits)

The keystream is: b'\xdb@)\xd3\xa7N\x9d:t\xe9\xd3\xa7N\x9d:t\xe9\xd3\xa7N\x9d:t\xe9\xd3\xa7N\x9d:t\xe9\xd3\xa7N\x9d:t\xe9\xd3\xa7N\x9d:t\xe9\xd3\xa7N\x9d:t\xe9\xd3\xa7N\x9d:t\xe9\xd3\xa7N\x9d:t\xe9\xd3\xa7N\x9d:t\xe9\xd3\xa7N\x9d:t\xe9\xd3\xa7N\x9d:t\xe9\xd3\xa7N\x9d:t\xe9\xd3\xa7N\x9d:t\xe9\xd3\xa7N\x9d:t\xe9\xd3\xa7N\x9d:t\xe9\xd3\xa7N\x9d:t\xe9\xd3\xa7N\x9d:t\xe9\xd3\xa7N\x9d:t\xe9\xd3\xa7N\x9d:t\xe9\xd3\xa7N\x9d:t\xe9\xd3\xa7'
The keystream is: [1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 

---

### __Exercise 3: Find the initial state of a correlated LFSR__

If an LFSR output is correlated with the keystream, find its initial state.

__(a) Explain how you find such an initial state and show the result.__

> To find the initial state we start by findind the keystream (found in exercise 2) which is the result of doing an $XOR$ between the $plaintext$ and the $ciphertext$.
>
> Because the functon $s(t)$ is $75\%$ correlated with $x_2$ and $x_3$, we start with LFSR2. We perform a brute-force search over all the 19-bit initial states that are not zero. For each initial state candidate we initialize LFSR2, generate the first `n` output bits ($x_2(t)$), and then compute the match rate with the keystream.
>
> From exercise 1 we know that the correct initial state should yield a match rate value around $0.75$, while the incorrect ones should yield a value around $0.50$. And since we can't know for sure which one is the correct initial value, we keep the top 5 initial values with highest matching rate value.

__(b) How many bits from the keystream did you use? What are the implications of using more or less bits from the keystream to find such correlations?__

> We used `n=1200` bits from the keystream because it was the value for which the matching rate stabilized around $0.75$. If we used less bits our code would have been faster performing but the matching rate value would have been near the $0.50$. So, more bits gives us more reliability and better results, but it does take longer to process.

In [32]:
# Plaintext bytes, bits
plaintext_bytes = plaintext.encode("utf-8")
L_bytes = len(plaintext_bytes)
L_bits  = 8 * L_bytes
print(L_bytes, L_bits)

152 1216


__(c) Provide or identify the code used to do it.__

In [33]:
def lfsr2_brute_force(n=2**19, best_total = 0, length = 100):
    """
    Comparar els bits entre lfsr2 i keystream --> 75%
    Només cal mirar els deg primers
    
    """
    x2_set = set()

    for len in range(n):

        x = rand_nonzero_bits(scipher.Cipher.deg2)
        x2_set.add(tuple(x))


      
    for x2_tuple in x2_set:
        x2 = list(x2_tuple)  

        lfsr = scipher.LFSR(x2, scipher.POLY_LFSR_2)
        x2_key = [lfsr.clock() for _ in range(length)]
        
        count = 0
        total = 0

        for i in range(length):
            if x2_key[i] == keystream_bits[i]:
                count += 1
        total = count / length
        
        if total > best_total:
            best_total = total
            best_init_state = x2_key

    
    return best_total, best_init_state
        

percentage, x2_initial_state = lfsr2_brute_force()

print (percentage, x2_initial_state)[:19]
    


KeyboardInterrupt: 

In [ ]:
def lfsr3_brute_force(n=2**21, best_total = 0, length = 100):
    """
    Comparar els bits entre lfsr3 i keystream --> 75%
    Només cal mirar els deg primers
    
    """
    x3_set = set()

    for len in range(n):

        x = rand_nonzero_bits(scipher.Cipher.deg3)
        x3_set.add(tuple(x))


      
    for x3_tuple in x3_set:
        x3 = list(x3_tuple)  

        lfsr = scipher.LFSR(x3, scipher.POLY_LFSR_3)
        x3_key = [lfsr.clock() for _ in range(length)]
        
        count = 0
        total = 0

        for i in range(length):
            if x3_key[i] == keystream_bits[i]:
                count += 1
        total = count / length
        
        if total > best_total:
            best_total = total
            best_init_state = x3_key

    
    return best_total, best_init_state
        

percentage, x3_initial_state = lfsr3_brute_force()

print(percentage, x3_initial_state)[:21]

1.0 [1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0]


---

### __Exercise 4: Define a strategy for a non-correlated LFSR__

If an LFSR’s output is not correlated to the keystream, define how you can find its initial state.
This strategy should improve on a brute-force search by leveraging the information obtained from correlated LFSR(s).

__(a) Explain how you can find the initial state for the LFSR(s) that are not correlated.__


> From the function $s(t)$ we know that $x_1$ decides which register output is used:  
> 
> - When $x_1=1$, $s=x_2$
> - When $x_1=0$, $s=x_3$
>
> There is no direct correlation between LFSR1 and the keystream, but when $x_2 \neq x_3$ we can directly know the value of $x_1$ without any calculation.
> Therefore, we only need to compute the bits where $x_2 = x_3$. We can check this by looking at the truth table from exercise 1:
> 
> For the rows where $x_2 \neq x_3$ we check how $s$ changes with $x_1$:
> - For $(x_2, x_3)=(1, 0)$: when $x_1=0 \Rightarrow s=0$, when $x_1=1 \Rightarrow s=1 \rightarrow s=x_1$
> - For $(x_2, x_3)=(0, 1)$: when $x_1=0 \Rightarrow s=1$, when $x_1=1 \Rightarrow s=0 \rightarrow s=\lnot x_1$
> 
> From this we conclude that when $x_2 \neq x_3$:
> - If $(x_2, x_3)=(1, 0) \rightarrow s=x_1$
> - If $(x_2, x_3)=(0, 1) \rightarrow s=\lnot x_1$
> 
> As we already know the keystream $s(t)$ and the outputs of $x_2(t)$ and $x_3(t)$, so we can use the same function to determine the missing bits of $x_1(t)$ and recover enough of its sequence to calculate the initial state of LFSR1.

---

### __Exercise 5: Find the key__

Find the key of the stream cipher.

__(a) Find the key for the stream cipher using the previous results and strategies.__


__(b) Provide or identify the code used to do it.__

In [ ]:
#We already know the keystream, and we have know more or less
x2
x3
keystream
x1 = []

#We know by the s equation that
for i, j in zip(x1,x2):
    if i == j:
        
    
    #So we only have to calculate when is a different case
    else:



IndentationError: expected an indented block after 'if' statement on line 9 (896445219.py, line 13)

__(c) Give an estimate of the computation time required to find it.__

---

### __Exercise 6: Decrypt the secret message__

Decrypt the secret message NN-secret-ciphertext.txt. with the key obtained earlier. Success or failure should be obvious.

__(a) Retrieve and show the secret message.__
